In [1]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F

import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [9]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

cuda


In [11]:
# Transforms allow us to process the image to the correct model specifications prior to training
transform = transforms.Compose(
    [transforms.RandomResizedCrop(224),
     transforms.ToTensor, # converts to from np array to tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 64

# Load our training and test sets via torchvision
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Dataloader allows us to minibatch our data when we pass it through our model
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# We do the same thing with the test set but we opt not to shuffle since we won't pass the test set more than once anyways
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
# Implement VGG16 Model:
class VGG16(nn.Module):
  def __init__(self):
    super(VGG16, self).__init__()

    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
    self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

    self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

    self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
    self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
    self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

    self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
    self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

    self.conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

    self.fc1 = nn.Linear(25088, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, 10)
  
  def forward(self, xb):
    xb = F.relu(self.conv1_1(xb))
    xb = F.relu(self.conv1_2(xb))
    xb = self.maxpool(xb)

    xb = F.relu(self.conv2_1(xb))
    xb = F.relu(self.conv2_2(xb))
    xb = self.maxpool(xb)

    xb = F.relu(self.conv3_1(xb))
    xb = F.relu(self.conv3_2(xb))
    xb = F.relu(self.conv3_3(xb))
    xb = self.maxpool(xb)

    xb = F.relu(self.conv4_1(xb))
    xb = F.relu(self.conv4_2(xb))
    xb = F.relu(self.conv4_3(xb))
    xb = self.maxpool(xb)

    xb = F.relu(self.conv5_1(xb))
    xb = F.relu(self.conv5_2(xb))
    xb = F.relu(self.conv5_3(xb))
    xb = self.maxpool(xb)

    xb = xb.reshape(xb.shape[0], -1)

    xb = F.relu(self.fc1(xb))
    xb = F.dropout(xb, 0.5)

    xb = F.relu(self.fc2(xb))
    xb = F.dropout(xb, 0.5)

    xb = F.relu(self.fc3(xb))

    return xb

In [13]:
model = VGG16()
model.to(dev)

VGG16(
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_2): Conv2d(

In [14]:
import numpy as np

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

def fit(epochs, model, opt, train_dl):
  for epoch in range(epochs):
    running_loss = 0
    model.train()
    for i, data in enumerate(train_dl, 0):
      inputs, labels = data
      opt.zero_grad()
      pred = model(inputs)
      loss = criterion(pred, labels)
      loss.backward()
      opt.step()
      running_loss += loss.item()
      if i % 2000 == 1999:
        print('[%d, %5d] loss: %.3f'%
              (epoch + 1, i + 1, running_loss/2000))
        running_loss = 0.0
      opt.zero_grad()

In [ ]:
fit(1, model, optimizer, trainloader)

In [ ]:
for epoch in range(2):
  running_loss = 0
  model.train()
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs = inputs.to(dev) # make the data a cuda tensor
    labels = labels.to(dev) # make the labels a cuda tensor
    optimizer.zero_grad()
    pred = model(inputs)
    loss = criterion(pred, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 2000 == 1999:
      print('[%d, %5d] loss: %.3f'%
            (epoch + 1, i + 1, running_loss/2000))
      running_loss = 0.0
    optimizer.zero_grad()